# Imports

In [77]:
import requests
import json
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
from ipyleaflet import Map, Marker


from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression

RUN_FROM = 'uni_wifi' #'bastion'

if RUN_FROM == 'bastion' : URL, HEADERS = 'http://fission:31001/', None
if RUN_FROM == 'uni_wifi': URL, HEADERS =  'http://172.26.135.52:9090/', {'HOST': 'fission'}

WEATHER_NUM_COL = ['UV', 'Min Temp', 'Max Temp', 'WindSpeed', 'Min Humid', 'Max Humid', 'Rain', 'Pan-Rain', 'Evapo-Rain']

# Functions

In [3]:
def weather_to_pd(station_id: str, start_year: int, end_year: int, verb=False) -> pd.DataFrame:
    resp_dict = json.loads(requests.get(URL+f'weather/{station_id}/{start_year}/{end_year}', headers=HEADERS).text)
    data = resp_dict['Data']
    if verb : print(f'Called weather api, fetched {len(data)} lines')
    return pd.DataFrame.from_records(data)

In [4]:
def get_stream_to_pd(api: str, station_id: str, size: int, radius_km: int, verb=False) -> pd.DataFrame:
    resp_dict = json.loads(requests.get(URL+api+f'/{station_id}/{size}/{radius_km}', headers=HEADERS).text)

    count=0
    status, token, new_data = resp_dict['Status'], resp_dict['Token'], resp_dict['Data']
    data = [new_data[i]['_source'] for i in range(len(new_data))]
    if verb : print(f'Called {api} api, fetched {len(new_data)} lines')


    while (status == 200) and (new_data != []) :
        count+=1
        resp_dict = json.loads(requests.get(URL+f'stream/'+token, headers=HEADERS).text)
        status, token, new_data = resp_dict['Status'], resp_dict['Token'], resp_dict['Data']
        if verb : print(f'Called stream {count} times, fetched {len(new_data)} new lines')
        data += [new_data[i]['_source'] for i in range(len(new_data))]

    if verb: print(f'Fetched a total of {len(data)}lines')
    return pd.DataFrame.from_records(data)

In [78]:
def clean_weather_pd(df_weather_full):
    df_weather = df_weather_full.copy()

    for col in WEATHER_NUM_COL:
        df_weather[col] = pd.to_numeric(df_weather[col])

    df_weather = df_weather.rename(columns={'Date':'date'})
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%d/%m/%Y').dt.date
    df_weather = df_weather.drop(columns=['created_at','source', 'Station Name'])

    return df_weather

In [ ]:
def clean_crime_df(df_crime_full):
    df_crime = df_crime_full.groupby(['reported_date', 'description_1'])['offence_count'].sum().reset_index()

    df_crime = df_crime.rename(columns={'reported_date':'date'})
    df_crime['date'] = pd.to_datetime(df_crime['date']).dt.date

    return df_crime

# Welcome to IISTGO

![title](img/isitsafe_banner.webp)

### General trends over the last year

### Does Weather impacts crime in your area

In [57]:
# Choose your position
center = (-38., 145.)
mm = Map(center=center, zoom=6)
marker = Marker(location=center, draggable=True)
mm.add(marker)
display(mm)

Map(center=[-38.0, 145.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [55]:
#Choose your radius for study
slider = widgets.FloatSlider(description='Radius in km')
slider.observe(compute, 'value')
slider.value = 50
widgets.VBox([slider])

In [76]:
# Summary of the study
LOCATION = marker.location
RADIUS = slider.value

resp = requests.get(URL+f'stations/{LOCATION[1]}/{LOCATION[0]}').json()
STATION_ID = resp['Data']['Station ID']

print(f'Chosen Coordinates: {LOCATION}')
print(f'Chosen study radius: {RADIUS} km')
print(f'Closest weather station name {resp["Data"]["Station Name"]}')
print(f'Study conducted over the last ten years')

Chosen Coordinates: [-38.0, 145.0]
Chosen study radius: 50.0 km
Closest weather station name FAWKNER BEACON
Study conducted over the last ten years


In [ ]:
# Fetching data and computing 
df_weather_full = weather_to_pd(station_id=STATION_ID, start_year =2014, end_year=2024,verb=True)
df_weather = clean_weather_pd(df_weather_full=)
df_crime_full = get_stream_to_pd(api='crime', station_id=STATION_ID, size=8000, radius_km=RADIUS, verb=True)
df_crime = clean_crime_df(df_crime_full=)
df = pd.merge(df_weather, df_crime, on='date', how='inner')

In [ ]:
lin_model_pers = LinearRegression()
lin_model_pers.fit(df_pers.drop(columns='offence_count').values, df_pers['offence_count'])
pd.DataFrame({'Predictors': df_pers.columns[:-1], 'Coefficient': lin_model_pers.coef_})